In [1]:
from ase.io import read
from ase import Atoms
import numpy as np
from icet import ClusterSpace
from icet.tools.structure_generation import  generate_sqs_from_supercells
from icet.input_output.logging_tools import set_log_config


i_atoms = read("POSCAR_I")
br_atoms = read("POSCAR_Br")

In [2]:
cell_diff = (i_atoms.cell - br_atoms.cell)

In [3]:
print("Br ", np.diag(br_atoms.cell))
print("I ", np.diag(i_atoms.cell))

Br  [5.86690399 6.02630628 6.08004591]
I  [6.24954534 6.36374306 6.41287239]


In [4]:
set_log_config(level='INFO')

species = [
    ["I", "Br"] if species == "Br" else [species]
    for species in br_atoms.get_chemical_symbols()
]

In [5]:
from pymatgen.io.ase import AseAtomsAdaptor

for i in range(1, 12):
    frac = i / 12
    new_cell = br_atoms.cell + frac * cell_diff
    
    new_atoms = Atoms(
        numbers=br_atoms.numbers,
        positions=br_atoms.positions,
        cell=new_cell,
        pbc=[True, True, True]
    )
    
    cs = ClusterSpace(new_atoms, [8.0, 4.0], species)
    target_concentrations = {'Br': (12-i)/12, 'I': i/12}
    
    supercell = new_atoms.repeat((4, 4, 4))
    
    sqs = generate_sqs_from_supercells(
        cluster_space=cs,
        supercells=[supercell],
        target_concentrations=target_concentrations,
        n_steps=60000
    )
    
    sqs = AseAtomsAdaptor().get_structure(sqs)
    sqs = sqs.get_sorted_structure()
    _ = sqs.to(filename=f"POSCAR_Br{12-i}_I{i}.vasp", fmt="vasp")

icet.orbit_list: INFO  Done getting matrix_of_equivalent_positions.
icet.orbit_list: INFO  Done getting neighbor lists.
icet.orbit_list: INFO  Transformation of matrix of equivalent positions to lattice neighbor format completed.
icet.orbit_list: INFO  Finished construction of orbit list.
icet.target_cluster_vector_annealing: INFO  Initializing target cluster vector annealing with 1 supercells
icet.target_cluster_vector_annealing: INFO  MC step 0/60000 (0 accepted trials, temperature 5.000), best score: 2.160
icet.target_cluster_vector_annealing: INFO  MC step 1000/60000 (903 accepted trials, temperature 1.861), best score: 1.062
icet.target_cluster_vector_annealing: INFO  MC step 2000/60000 (1810 accepted trials, temperature 1.546), best score: 1.062
icet.target_cluster_vector_annealing: INFO  MC step 3000/60000 (2692 accepted trials, temperature 1.362), best score: 1.062
icet.target_cluster_vector_annealing: INFO  MC step 4000/60000 (3561 accepted trials, temperature 1.231), best sco